In [1]:
import os

In [2]:
%pwd

'/Users/chaky/Desktop/data science/projects/end-to-end-machine-learning-project-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/chaky/Desktop/data science/projects/end-to-end-machine-learning-project-with-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def transform_data(self):
        # Load the data
        data = pd.read_csv(self.config.data_path)

        # Shuffle the data
        data = data.sample(frac=1, random_state=42).reset_index(drop=True)

        # Define the target variable
        target_variable = 'quality'

        # Split the data into features (X) and the target variable (y)
        #X = data.drop(target_variable, axis=1)
        #y = data[target_variable]

        # Split the data into training and test sets with randomization (0.75, 0.25) split.
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        # Apply StandardScaler only to the features (X)
        #scaler = StandardScaler()
        #X_train_scaled = scaler.fit_transform(X_train)
        #X_test_scaled = scaler.transform(X_test)

        # Apply PCA (Principal Component Analysis) if needed
        # pca = PCA(n_components=2)  # You can specify the number of components as needed
        # X_train_pca = pca.fit_transform(X_train_scaled)
        # X_test_pca = pca.transform(X_test_scaled)
        train, test = train_test_split(data, test_size=0.25, random_state=42)
        X_train=train.drop(target_variable, axis=1)
        y_train=train[target_variable]

        X_test=test.drop(target_variable, axis=1)
        y_test=test[target_variable]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Save the transformed data
        train_data = pd.DataFrame(X_train_scaled, columns=X_train.columns)
        test_data = pd.DataFrame(X_test_scaled, columns=X_test.columns)

        # Add the target variable to the train and test datasets
        train_data[target_variable] = y_train
        test_data[target_variable] = y_test

        # Save the train and test datasets as CSV files
        train_data.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test_data.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        # Save the transformed data
        #train_data = pd.DataFrame(X_train_scaled, columns=X.columns)
        #test_data = pd.DataFrame(X_test_scaled, columns=X.columns)
        # Add the target variable to the train and test datasets

        #train_data[target_variable] = y_train
        #test_data[target_variable] = y_test
        # Save the train and test datasets as CSV files

        #train_data.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        #test_data.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Transformed data with StandardScaler and saved to CSV files with randomization")
        logger.info(train_data.shape)
        logger.info(test_data.shape)

        print(train_data.shape)
        print(test_data.shape)

In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform_data()
except Exception as e:
    raise e

[2023-10-28 14:36:43,986: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-28 14:36:43,991: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-28 14:36:43,995: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-28 14:36:43,997: INFO: common: created directory at: artifacts]
[2023-10-28 14:36:43,997: INFO: common: created directory at: artifacts/data_transformation]
[2023-10-28 14:36:44,049: INFO: 2466589463: Transformed data with StandardScaler and saved to CSV files with randomization]
[2023-10-28 14:36:44,049: INFO: 2466589463: (1199, 12)]
[2023-10-28 14:36:44,050: INFO: 2466589463: (400, 12)]
(1199, 12)
(400, 12)
